In [ ]:
!pip install uproot

In [ ]:
!pip install uproot awkward vector numpy scipy

In [ ]:
import uproot
import awkward as ak
import vector
import numpy as np
from tqdm import tqdm

vector.register_awkward()

# ===============================
# CONFIG
# ===============================
INPUT_ROOT  = "/kaggle/input/cms2011b-requiredbranches/merged.root"
OUTPUT_ROOT = "stage3_event_shapes.root"

JET_PT_MIN = 30.0  # GeV

# ===============================
# OPEN FILE
# ===============================
tree = uproot.open(INPUT_ROOT)["Events"]
branches = tree.keys()

def find_branch(prefix, suffix):
    for b in branches:
        if b.startswith(prefix) and b.endswith(suffix):
            return b
    raise RuntimeError(f"Missing branch: {prefix}*{suffix}")

# ===============================
# JET BRANCHES
# ===============================
b_pt   = find_branch("recoPFJets_ak5PFJets__RECO", "pt_")
b_eta  = find_branch("recoPFJets_ak5PFJets__RECO", "eta_")
b_phi  = find_branch("recoPFJets_ak5PFJets__RECO", "phi_")
b_mass = find_branch("recoPFJets_ak5PFJets__RECO", "mass_")

arrays = tree.arrays([b_pt, b_eta, b_phi, b_mass], library="ak")

jets = ak.zip(
    {
        "pt": arrays[b_pt],
        "eta": arrays[b_eta],
        "phi": arrays[b_phi],
        "mass": arrays[b_mass],
    },
    with_name="Momentum4D"
)

jets = jets[jets.pt > JET_PT_MIN]
njets = ak.num(jets, axis=1)

# ===============================
# MOMENTUM TENSORS
# ===============================
def safe_norm(x):
    return ak.where(x > 0, x, 1.0)

px, py, pz = jets.px, jets.py, jets.pz
p2 = px**2 + py**2 + pz**2
pt2 = px**2 + py**2

norm3 = safe_norm(ak.sum(p2, axis=1))
normT = safe_norm(ak.sum(pt2, axis=1))

# 3D tensor
Sxx = ak.sum(px*px, axis=1) / norm3
Syy = ak.sum(py*py, axis=1) / norm3
Szz = ak.sum(pz*pz, axis=1) / norm3
Sxy = ak.sum(px*py, axis=1) / norm3
Sxz = ak.sum(px*pz, axis=1) / norm3
Syz = ak.sum(py*pz, axis=1) / norm3

# ===============================
# EIGENVALUES (3×3)
# ===============================
eig3 = []
for i in range(len(Sxx)):
    M = np.array([
        [Sxx[i], Sxy[i], Sxz[i]],
        [Sxy[i], Syy[i], Syz[i]],
        [Sxz[i], Syz[i], Szz[i]],
    ])
    vals = np.linalg.eigvalsh(M)
    vals = np.clip(vals, 0, None)
    vals /= np.sum(vals) if np.sum(vals) > 0 else 1.0
    eig3.append(vals)

eig3 = np.sort(np.array(eig3), axis=1)[:, ::-1]

sphericity = 1.5 * (eig3[:,1] + eig3[:,2])
aplanarity = 1.5 * eig3[:,2]

# ===============================
# THRUST (robust approximation)
# ===============================
p_mag = ak.sum(jets.p, axis=1)
thrust = ak.where(
    p_mag > 0,
    ak.max(ak.sum(abs(px), axis=1), axis=0) / p_mag,
    0.0
)

# ===============================
# FOX–WOLFRAM H2
# ===============================
pairs = ak.combinations(jets, 2)
dot = (
    pairs["0"].px * pairs["1"].px +
    pairs["0"].py * pairs["1"].py +
    pairs["0"].pz * pairs["1"].pz
)
den = pairs["0"].p * pairs["1"].p
costh = ak.where(den > 0, dot / den, 0.0)

P2 = 0.5 * (3*costh**2 - 1)

H2 = ak.sum(pairs["0"].p * pairs["1"].p * P2, axis=1)
H0 = ak.sum(jets.p, axis=1)**2

foxWolfram_H2 = ak.where(H0 > 0, H2 / H0, 0.0)

# ===============================
# TRANSVERSE SHAPES
# ===============================
SxxT = ak.sum(px*px, axis=1) / normT
SyyT = ak.sum(py*py, axis=1) / normT
SxyT = ak.sum(px*py, axis=1) / normT

eig2 = []
for i in range(len(SxxT)):
    M = np.array([[SxxT[i], SxyT[i]], [SxyT[i], SyyT[i]]])
    vals = np.linalg.eigvalsh(M)
    vals = np.clip(vals, 0, None)
    vals /= np.sum(vals) if np.sum(vals) > 0 else 1.0
    eig2.append(vals)

eig2 = np.sort(np.array(eig2), axis=1)[:, ::-1]

transverse_sphericity = 2.0 * eig2[:,1]
isotropy = eig2[:,1] / safe_norm(eig2[:,0])

# ===============================
# OUTPUT (FIXED)
# ===============================
out = {
    "sphericity": np.clip(sphericity, 0, 1).astype(np.float32),
    "aplanarity": np.clip(aplanarity, 0, 0.5).astype(np.float32),
    "thrust": np.clip(
        ak.to_numpy(thrust),
        0, 1
    ).astype(np.float32),
    "foxWolfram_H2": ak.to_numpy(foxWolfram_H2).astype(np.float32),
    "isotropy": np.clip(
        ak.to_numpy(isotropy),
        0, 1
    ).astype(np.float32),
    "transverse_sphericity": np.clip(
        transverse_sphericity,
        0, 1
    ).astype(np.float32),
}


with uproot.recreate(OUTPUT_ROOT) as fout:
    fout["Events"] = out

print(f"\n✅ Stage-3 Event Shapes written → {OUTPUT_ROOT}")


In [ ]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------
# Load ROOT file
# ---------------------------------
FILE = "/kaggle/working/stage3_event_shapes.root"

f = uproot.open(FILE)
tree = f["Events"]
arr = tree.arrays(library="ak")

print("\nTotal events:", len(arr))
print("Available branches:", list(arr.fields))

# ---------------------------------
# Basic statistics
# ---------------------------------
print("\nBasic stats:")
for k in arr.fields:
    print(
        f"{k:22s} min/max:",
        float(ak.min(arr[k])),
        float(ak.max(arr[k]))
    )

# ---------------------------------
# Physics sanity checks
# ---------------------------------
print("\nPhysics sanity checks:")

print("Sphericity outside [0,1]:",
      ak.sum((arr.sphericity < 0) | (arr.sphericity > 1)))

print("Aplanarity outside [0,0.5]:",
      ak.sum((arr.aplanarity < 0) | (arr.aplanarity > 0.5)))

print("Thrust outside [0,1]:",
      ak.sum((arr.thrust < 0) | (arr.thrust > 1)))

print("Isotropy outside [0,1]:",
      ak.sum((arr.isotropy < 0) | (arr.isotropy > 1)))

print("Transverse sphericity outside [0,1]:",
      ak.sum((arr.transverse_sphericity < 0) |
             (arr.transverse_sphericity > 1)))

# ---------------------------------
# Event masks
# ---------------------------------
mask_valid = (
    (arr.sphericity > 0) &
    (arr.transverse_sphericity > 0)
)

print("\nEvent counts:")
print("Valid shape events:", ak.sum(mask_valid))

# ---------------------------------
# 1) Sphericity
# ---------------------------------
plt.figure()
plt.hist(ak.to_numpy(arr.sphericity), bins=100)
plt.xlabel("Sphericity")
plt.ylabel("Events")
plt.title("Sphericity (all events)")
plt.show()

# ---------------------------------
# 2) Aplanarity
# ---------------------------------
plt.figure()
plt.hist(ak.to_numpy(arr.aplanarity), bins=100)
plt.xlabel("Aplanarity")
plt.ylabel("Events")
plt.title("Aplanarity (all events)")
plt.show()

# ---------------------------------
# 3) Thrust
# ---------------------------------
plt.figure()
plt.hist(ak.to_numpy(arr.thrust), bins=100)
plt.xlabel("Thrust")
plt.ylabel("Events")
plt.title("Thrust")
plt.show()

# ---------------------------------
# 4) Fox–Wolfram H2
# ---------------------------------
plt.figure()
plt.hist(
    ak.to_numpy(arr.foxWolfram_H2),
    bins=100,
    range=(-0.3, 1.0)
)
plt.xlabel("Fox–Wolfram H2")
plt.ylabel("Events")
plt.title("Fox–Wolfram Moment H2")
plt.show()

# ---------------------------------
# 5) Isotropy
# ---------------------------------
plt.figure()
plt.hist(ak.to_numpy(arr.isotropy), bins=100)
plt.xlabel("Isotropy")
plt.ylabel("Events")
plt.title("Event Isotropy (2D)")
plt.show()

# ---------------------------------
# 6) Transverse Sphericity
# ---------------------------------
plt.figure()
plt.hist(ak.to_numpy(arr.transverse_sphericity), bins=100)
plt.xlabel("Transverse Sphericity")
plt.ylabel("Events")
plt.title("Transverse Sphericity")
plt.show()

# ---------------------------------
# Correlations
# ---------------------------------
plt.figure()
plt.scatter(
    ak.to_numpy(arr.sphericity[mask_valid]),
    ak.to_numpy(arr.transverse_sphericity[mask_valid]),
    s=1,
    alpha=0.3
)
plt.xlabel("Sphericity")
plt.ylabel("Transverse Sphericity")
plt.title("Sphericity vs Transverse Sphericity")
plt.show()

plt.figure()
plt.scatter(
    ak.to_numpy(arr.thrust[mask_valid]),
    ak.to_numpy(arr.sphericity[mask_valid]),
    s=1,
    alpha=0.3
)
plt.xlabel("Thrust")
plt.ylabel("Sphericity")
plt.title("Thrust vs Sphericity")
plt.show()

print("\n✔ Stage-3 Event Shape EDA complete.")


In [ ]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

FILE = "/kaggle/working/stage3_event_shapes.root"

with uproot.open(FILE) as f:
    tree = f["Events"]
    df = tree.arrays(library="pd")

print(df.shape)
df.head()
